Q2. Re-implement the same model using some built-in functions and classes from PyTorch.
And now using two different targets: Apples and Oranges

In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
# Input (temp, rainfall, humidity)
input = np.array([[75, 72, 40], 
                            [100, 89, 68], 
                            [85, 120, 62], 
                            [95, 55, 39], 
                            [65, 90, 75], 
                            [83, 72, 56], 
                            [81, 88, 75], 
                            [78, 114, 99], 
                            [98, 51, 57], 
                            [69, 96, 71], 
                            [79, 76, 54], 
                            [72, 77, 84], 
                            [67, 125, 67], 
                            [100, 53, 46], 
                            [70, 107, 60]], 
                            dtype='float32')

# Targets (apples, oranges)
target = np.array([[54, 75], 
                              [79, 121], 
                              [105, 130], 
                              [33, 39], 
                              [101, 110],
                              [45, 79], 
                              [85, 92], 
                              [110, 122], 
                              [36, 35], 
                              [95, 114], 
                              [57, 69], 
                              [82, 100], 
                              [118, 134], 
                              [20, 38], 
                              [102, 120]], 
                              dtype='float32')

inputs = torch.from_numpy(input)
targets = torch.from_numpy(target)

In [3]:
from torch.utils.data import TensorDataset
# Defining dataset

train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 75.,  72.,  40.],
         [100.,  89.,  68.],
         [ 85., 120.,  62.]]), tensor([[ 54.,  75.],
         [ 79., 121.],
         [105., 130.]]))

In [4]:
from torch.utils.data import DataLoader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
# Model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3115, -0.4072,  0.5100],
        [ 0.2019,  0.4902,  0.2081]], requires_grad=True)
Parameter containing:
tensor([-0.1728,  0.2185], requires_grad=True)


In [5]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3115, -0.4072,  0.5100],
         [ 0.2019,  0.4902,  0.2081]], requires_grad=True),
 Parameter containing:
 tensor([-0.1728,  0.2185], requires_grad=True)]

In [6]:
# Import nn.functional
import torch.nn.functional as F
# Defining loss function
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)
# Defining optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

tensor(2498.3804, grad_fn=<MseLossBackward0>)


In [7]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [8]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 396.3764
Epoch [20/100], Loss: 200.4623
Epoch [30/100], Loss: 341.7484
Epoch [40/100], Loss: 368.2802
Epoch [50/100], Loss: 205.0855
Epoch [60/100], Loss: 118.4180
Epoch [70/100], Loss: 156.6588
Epoch [80/100], Loss: 109.9825
Epoch [90/100], Loss: 134.5971
Epoch [100/100], Loss: 105.8490


In [9]:
# Generating predictions
preds = model(inputs)
preds

tensor([[ 51.6931,  71.6521],
        [ 75.7519,  93.8026],
        [ 93.2053, 124.8305],
        [ 35.8974,  50.9874],
        [ 90.5856, 103.5871],
        [ 61.4342,  75.8993],
        [ 85.2725,  98.7364],
        [118.8671, 133.3565],
        [ 46.0918,  52.9178],
        [ 90.0755, 107.3379],
        [ 63.3166,  79.8393],
        [ 87.8711,  92.7532],
        [104.4715, 134.7674],
        [ 38.6091,  50.6101],
        [ 88.0759, 114.0526]], grad_fn=<AddmmBackward0>)

In [10]:
# Comparing with targets
k=0
for i in preds:
  print("prediction : ",i," Target : ",targets[k])
  k+=1

prediction :  tensor([51.6931, 71.6521], grad_fn=<UnbindBackward0>)  Target :  tensor([54., 75.])
prediction :  tensor([75.7519, 93.8026], grad_fn=<UnbindBackward0>)  Target :  tensor([ 79., 121.])
prediction :  tensor([ 93.2053, 124.8305], grad_fn=<UnbindBackward0>)  Target :  tensor([105., 130.])
prediction :  tensor([35.8974, 50.9874], grad_fn=<UnbindBackward0>)  Target :  tensor([33., 39.])
prediction :  tensor([ 90.5856, 103.5871], grad_fn=<UnbindBackward0>)  Target :  tensor([101., 110.])
prediction :  tensor([61.4342, 75.8993], grad_fn=<UnbindBackward0>)  Target :  tensor([45., 79.])
prediction :  tensor([85.2725, 98.7364], grad_fn=<UnbindBackward0>)  Target :  tensor([85., 92.])
prediction :  tensor([118.8671, 133.3565], grad_fn=<UnbindBackward0>)  Target :  tensor([110., 122.])
prediction :  tensor([46.0918, 52.9178], grad_fn=<UnbindBackward0>)  Target :  tensor([36., 35.])
prediction :  tensor([ 90.0755, 107.3379], grad_fn=<UnbindBackward0>)  Target :  tensor([ 95., 114.])
pr

In [11]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)